In [1]:
import numpy as np
import matplotlib 
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import random
import pandas as pd
from statistics import mean 
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt

In [71]:
# Defining a function to train SVM on a model and compute the error on any specified dataset
def regul_regression_analysis(data_training,data_testing,plus_one_label,reg_param,minus_one_label=10):
    """
    Inputs-
    data_training: A Pandas dataframe of Nx3 to be used as a training set where the first column is the specific digit, second column
                   is the intensity and third is symmetry
    data_testing: A Pandas dataframe of Nx3 to be used as a test set where the first column is the specific digit, second column
                   is the intensity and third is symmetry
    plus_one_label: digit to be assigned the label +1
    minus_one_label: digit to be assigned -1 label, only valid if we are doing one digit versus another one
                     with all others being disregarded (by default it is 10)
    reg_param: Regularization parameter
    
    Output-
    error_regression - computes the error of the model on the dataset specified for testing
    """
    # Preparing the Dataset
    if 0<=minus_one_label<10:
        data_training = data_training.loc[(data_training['digit']==plus_one_label) | (data_training['digit']==minus_one_label)]
        data_testing = data_testing.loc[(data_testing['digit']==plus_one_label) | (data_testing['digit']==minus_one_label)] 
    train_data = data_training.values
    test_data = data_testing.values
    y_train = (np.where(data_training['digit']==plus_one_label, 1, -1)).reshape(-1,1)
    y_test = (np.where(data_testing['digit']==plus_one_label, 1, -1)).reshape(-1,1)
    X_train = train_data[:,1:-1]
    X_test = test_data[:,1:-1]
    # Fitting the Regularized Linear Regression Model
    n = X_train.shape[0]
    n_test = X_test.shape[0]
    d = X_train.shape[1]
    X_train_design = np.append(np.ones((n,1)),X_train,axis=1)  
    X_test_design = np.append(np.ones((n_test,1)),X_test,axis=1)
    a = np.linalg.inv(np.matmul(np.transpose(X_train_design),X_train_design) + (reg_param*np.identity(d+1)))
    b = np.dot(np.transpose(X_train_design),y_train)
    final_weight = (np.dot(a,b)).reshape(-1,1)
    
    # Making predictions and computing the error
    pred_regress = (np.dot(X_test_design,final_weight)).reshape(-1,1)
    err_vec = (pred_regress.reshape(-1,))*(y_test.reshape(-1,))
    error_regress = np.sum(err_vec < 0)/n_test
    return error_regress

In [77]:
# Importing Digit Training and Test datasets
train_data = pd.read_csv('digit_identification_training.txt', sep="\t")
test_data = pd.read_csv('digit_identification_testing.txt', sep="\t")

In [78]:
# Regularization problems
# Problem 7
Ein_vec = []
reg_param = 1
digits_vec = [5,6,7,8,9]
for i in digits_vec:
    Ein_vec.append(regul_regression_analysis(train_data,train_data,i,reg_param,minus_one_label=10))
print("The minimum in sample error among the digits in digits_vec is observed for : ",digits_vec[Ein_vec.index(min(Ein_vec))])

The minimum in sample error among the digits in digits_vec is observed for :  8


In [80]:
# Problem 8
Ein_overall_vec_wot = []
Eout_overall_vec_wot = []
digits_vec = [0,1,2,3,4,5,6,7,8,9]
for i in digits_vec:
    Ein_overall_vec_wot.append(regul_regression_analysis(train_data,train_data,i,reg_param,minus_one_label=10))
    Eout_overall_vec_wot.append(regul_regression_analysis(train_data,test_data,i,reg_param,minus_one_label=10))
print("The in sample error vector without transformation : ",Ein_overall_vec_wot)
print("The out-of-sample error vector without transformation: ",Eout_overall_vec_wot)

train_data['x1_sq'] = train_data['intensity']**2
train_data['x2_sq'] = train_data['symmetry']**2
train_data['x1_x2'] = train_data['intensity']*train_data['symmetry']
test_data['x1_sq'] = test_data['intensity']**2
test_data['x2_sq'] = test_data['symmetry']**2
test_data['x1_x2'] = test_data['intensity']*test_data['symmetry']

Ein_overall_vec_wt = []
Eout_overall_vec_wt = []
for i in digits_vec:
    Ein_overall_vec_wt.append(regul_regression_analysis(train_data,train_data,i,reg_param,minus_one_label=10))
    Eout_overall_vec_wt.append(regul_regression_analysis(train_data,test_data,i,reg_param,minus_one_label=10))
print("The in sample error vector with transformation: ",Ein_overall_vec_wt)
print("The out of sample error vector with transformation : ",Eout_overall_vec_wt)

In [75]:
# Problem 9
Eout_vec=[]
digits_vec = [0,1,2,3,4]
for i in digits_vec:
    Eout_vec.append(regul_regression_analysis(train_data,test_data,i,reg_param,minus_one_label=10))
print("The observed out of sample error vector is: ", Eout_vec)
print("The minimum in sample error among the digits in digits_vec is observed for : ",digits_vec[Eout_vec.index(min(Eout_vec))])

The observed out of sample error vector is:  [0.1111111111111111, 0.02242152466367713, 0.09865470852017937, 0.08271051320378675, 0.09965122072745392]
The minimum in sample error among the digits in digits_vec is observed for :  1


In [87]:
# Problem 10
reg_params_vec = [0.01,1]
Ein_reg_vec = []
Eout_reg_vec = []
for j in reg_params_vec:
    Ein_reg_vec.append(regul_regression_analysis(train_data,train_data,plus_one_label=1,reg_param=j,minus_one_label=5))
    Eout_reg_vec.append(regul_regression_analysis(train_data,test_data,plus_one_label=1,reg_param=j,minus_one_label=5))
print("The in sample error vector for the two values of regularization parameter is: ", Ein_reg_vec)
print("The out-of-sample error vector for the two values of regularization parameter is: ", Eout_reg_vec)

The in sample error vector for the two values of regularization parameter is:  [0.005124919923126201, 0.005124919923126201]
The out-of-sample error vector for the two values of regularization parameter is:  [0.02830188679245283, 0.025943396226415096]


As we move from lambda=1 to lambda=0.01 we can clearly see that overfitting occurs as the testing error increases with no change in training error. This implies that 0.01 is not enough strength to overpower the complicated set of features we are using and we need to use a larger value.

In [ ]:
# SVM problems

# Problem 11
# Transforming x1,x2 into z1 and z2
def transform_fun(a,b):
    return (b**2 - 2*a - 1,a**2 - 2*b + 1)
x = [(1,0),(0,1),(0,-1),(-1,0),(0,2),(0,-2),(-2,0)]
y = [-1,-1,-1,1,1,1,1]
z1 = []
z2 = []
for i in x:
    temp = transform_fun(i[0],i[1])
    z1.append(temp[0])
    z2.append(temp[1])

# Plotting the points
fig = plt.figure() 
ax = fig.add_subplot(111) 
ax.set_xlim(-4, 4)
ax.set_title('Points in z1-z2 system')
ax.set_ylabel('Z2')
ax.set_xlabel('Z1')
ax.scatter(z1, z2, c=[1, 1, 1, 2,2,2,2], marker='^')
plt.show()

From the location of points it is pretty clear that z1-0.5=0 line clearly separates the points into two categories as dictated in the problem.

In [102]:
# Problem 12
data_svm = (np.array([z1,z2])).reshape(-1,2)
resp_svm = (np.array([y])).reshape(-1,)
clf = svm.SVC(C=1e20, degree=2, kernel='poly',gamma=1,coef0=1)
svm_fit = clf.fit(data_svm, resp_svm)
print("The number of support vectors are: ",(svm_fit.support_vectors_).shape[0])

In [115]:
# Comparing standalone RBFs with RBF kernels being used in SVM 
def gen_data_rbf(llim,ulim,N):
    
    """
    llim: a scalar defining the lower limit on the x and y axis
    ulim: a scalar defining the upper limit on the x and y axis
    N: Total number of data points
    Output: Returns a numpy array of Nx3 where the first and second column 
            consists of N points generated uniformly between llim and ulim and the 3th column being 
            the labels given as per the target function
    """
    x1 = (np.random.uniform(llim,ulim,N)).reshape(N,1)
    x2 = (np.random.uniform(llim,ulim,N)).reshape(N,1)
    data_partial = np.append(x1,x2,axis=1)    
    target = x2 - x1 + (0.25*np.sin(np.pi*x1))
    mask = target>0
    category_values = np.where(mask,np.ones((N,1)),-1)
    data_complete = np.append(data_partial,category_values,axis=1)
    
    return data_complete

def vis_target_classification(data_complete):
    """
    data_complete: Returns a numpy array of Nx3 where the first and second column 
                   consists of N points generated uniformly between llim and ulim and the 3th column being 
                   the labels given as per the target function
    """
    
    fig = plt.figure() 
    ax = fig.add_subplot(111) 
    ax.set_xlim(-1, 1)
    ax.set_title('Points separated by f=x2-x1 + 0.25*sin(pi*x1)')
    ax.set_ylabel('x1')
    ax.set_xlabel('x2')
    ax.scatter(data_complete[:,0],data_complete[:,1], c=data_complete[:,2], marker='^')
    plt.show()
    return()

In [117]:
# Specifying the problem parameters
llim=-1
ulim=1
N = 100
iters = 10000

In [120]:
# Problem 13: Fitting the SVM to compute the proportion of misspecification for hard margins
clf = svm.SVC(C=1e20,kernel='rbf',gamma=1.5)
iter_incorrect = 0
for i in range(iters):
    data_temp = gen_data_rbf(llim,ulim,N)
    svm_fit = clf.fit(data_temp[:,0:2], (data_temp[:,2]).reshape(-1,))
    pred_svm = (svm_fit.predict(data_temp[:,0:2])).reshape(-1,1)
    error_SVM = np.mean(pred_svm!=((data_temp[:,2]).reshape(-1,1)))
    if error_SVM!=0:
        iter_incorrect = iter_incorrect + 1
print("The proportion of times hard margin SVM fails to classify all the in sample points correctly is: ",iter_incorrect/iters)

The proportion of times hard margin SVM fails to classify all the in sample points correctly is:  0.0


In [298]:
# Group of functions estimate the centres in an RBF kernel using Lloyd's algorithm and 
# compute the weights using Linear regression
def computing_clusters(data_partial,centres):
    """
    Inputs
    data_partial: Nxr numpy array containing only the design columns(excluding 1s and responses/categories)
    centres: Kxr numpy array where K corresponds to the number of clusters and r is the dimension of input 
             space
             
    Output
    cluster_matrix: K X N numpy array whose (i,j) point corresponds to the following
                    - 0 if jth point doesn't belongs to the cluster
                    - i if jth point belongs to the cluster 
    """
    K = centres.shape[0]
    N = data_partial.shape[0]
    cluster_matrix = np.zeros((K,N))
    for i in range(N):
        dist_point = (np.sum(np.power(((data_partial[i,:]).reshape(-1,)) - centres,2),axis=1)).reshape(K,)
        cluster_matrix[:,i] = dist_point
        mask = cluster_matrix[:,i]==np.amin(cluster_matrix[:,i])
        cluster_matrix[:,i]= (np.where(mask,np.argmin(cluster_matrix[:,i])+1,0)).reshape(K,)
    return cluster_matrix

def updating_centres(data_partial,cluster_matrix):
    """
    Input
    data_partial: Nxr numpy array containing only the design columns(excluding 1s and responses/categories)
    cluster_matrix: K X N numpy array whose (i,j) point corresponds to the following
                    - 0 if jth point doesn't belongs to the cluster
                    - i if jth point belongs to the cluster
    
    Output
    updated_centres: A numpy array of K x r giving the updated weights as the means of computed clusters
    """
    K = cluster_matrix.shape[0]
    r = data_partial.shape[1]
    updated_centres = np.zeros((K,r)) 
    for i in range(K):
        inds = np.argwhere(cluster_matrix==i)
        if inds.size==0: # test for an insolvable clustering situation
            updated_centres = np.ones((K,r)) 
            break
        else:
            inds = inds[:,1]
            updated_centres[i,:] = (np.mean(data_partial[inds,:],axis=0)).reshape(r,)
    return updated_centres

def lloyd_algorithm(data_partial,centres):
    """
    Inputs
    data_partial: Nxr numpy array containing only the design columns(excluding 1s and responses/categories)
    centres: Kxr numpy array where K corresponds to the number of clusters and r is the dimension of input 
             space
             
    Output
    final_centres: K X r numpy array containing the final centres for each of the clusters
    """
    cm = computing_clusters(data_partial,centres)
    cm1 = cm+10
    while (np.all(cm==cm1)!=True): 
        centres = updating_centres(data_partial,cm)
        if np.all(centres==1)==True:
            break
        else:
            cm1 = computing_clusters(data_partial,centres)
        if np.all(cm==cm1)==True:
            break
        else:
            cm=cm1
    return centres

def rbf_kerel_weights(data_partial,response_categories_train,gamma,centres):
    """
    Inputs
    data_partial: Nxr numpy array containing only the design columns(excluding 1s and responses/categories)
    response_categories_train: (N,1) or (N,) numpy array containing the response categories based on target function
    gamma: scaling factor for the RBF kernel
    centres: Kxr numpy array where K corresponds to the number of clusters and r is the dimension of input 
             space
             
    Output
    final_weights: numpy array of shape (K+1,1) containing the estimated weights associated with each of the cluster 
                   terms
    """
    K = centres.shape[0]
    N = data_partial.shape[0]
    design_matrix = np.zeros((N,K))
    for i in range(N):
        dist_point = np.exp(-gamma*((np.sum(np.power(((data_partial[i,:]).reshape(-1,)) - centres,2),axis=1)).reshape(K,)))
        design_matrix[i,:] = dist_point
    design_matrix = np.append(np.ones((N,1)),design_matrix,axis=1)
    a = np.linalg.inv(np.matmul(np.transpose(design_matrix),design_matrix))
    b = np.dot(np.transpose(design_matrix),response_categories_train.reshape(-1,1))
    final_weights = np.dot(a,b)
    return final_weights

def pred_rbf_kernel(data_partial_test,response_categories_test,final_centres,final_weights,gamma):
    """
    Inputs
    data_partial_test: Nxr numpy array containing only the design columns(excluding 1s and responses/categories)
    response_categories_test: (N,1) or (N,) numpy array containing the response categories based on target function
    gamma: scaling factor for the RBF kernel
    final_centres: Kxr numpy array where K corresponds to the number of clusters and r is the dimension of input 
             space
    final_weights: numpy array of shape (K+1,1) containing the estimated weights associated with each of 
                   the cluster terms
    
    Output
    error: total error made by the RBF using the binary classification error measure
    """
    K = final_centres.shape[0]
    N = data_partial_test.shape[0]
    design_matrix_test = np.zeros((N,K))
    for i in range(N):
        dist_point = np.exp(-gamma*((np.sum(np.power(((data_partial_test[i,:]).reshape(-1,)) - final_centres,2),axis=1)).reshape(K,)))
        design_matrix_test[i,:] = dist_point
    design_matrix_test = np.append(np.ones((N,1)),design_matrix_test,axis=1)
    pred_rbf_rval = (np.dot(design_matrix_test,final_weights)).reshape(-1,)
    mask = pred_rbf_rval>0
    pred_category_values = np.where(mask,1,-1)
    error_rbf = np.mean(pred_category_values!=(response_categories_test.reshape(-1,1)))
    return error_rbf

In [299]:
# Problem 14: K=9 gamma=1.5
K = 9
iters = 1000
gamma=1.5
SVM_better_than_rbf = 0
for i in range(iters):
    # Generating the Dataset for ith iteration
    error_train_SVM=0.1
    cond=True
    while (error_train_SVM!=0) or (cond==True):
        data_temp_train = gen_data_rbf(llim,ulim,N)
        data_temp_test = gen_data_rbf(llim,ulim,N)
        svm_fit = clf.fit(data_temp_train[:,0:2], (data_temp_train[:,2]).reshape(-1,))
        pred_svm_train = (svm_fit.predict(data_temp_train[:,0:2])).reshape(-1,1)
        error_train_SVM = np.mean(pred_svm_train!=((data_temp_train[:,2]).reshape(-1,1)))
        
        response_categories_train = data_temp_train[:,2]
        response_categories_test = data_temp_test[:,2]
        centres_init = (np.random.uniform(llim,ulim,2*K)).reshape(K,-1)
        final_centres = lloyd_algorithm(data_temp_train[:,0:2],centres_init)
        cond = np.all(final_centres==1)
        
    pred_svm = (svm_fit.predict(data_temp_test[:,0:2])).reshape(-1,1)
    error_test_SVM = np.mean(pred_svm!=((data_temp_test[:,2]).reshape(-1,1)))
    final_weights = rbf_kerel_weights(data_temp_train[:,0:2],response_categories_train,gamma,final_centres)
    error_test_rbf = pred_rbf_kernel(data_temp_test[:,0:2],response_categories_test,final_centres,final_weights,gamma)
    
    if error_test_rbf>error_test_SVM:
        SVM_better_than_rbf = SVM_better_than_rbf + 1
SVM_better_than_rbf/iters

1.0

In [301]:
# Problem 15: K=12 gamma=1.5
K = 12
iters = 10000
gamma=1.5
SVM_better_than_rbf = 0
for i in range(iters):
    # Generating the Dataset for ith iteration
    error_train_SVM=0.1
    cond=True
    while (error_train_SVM!=0) or (cond==True):
        data_temp_train = gen_data_rbf(llim,ulim,N)
        data_temp_test = gen_data_rbf(llim,ulim,N)
        svm_fit = clf.fit(data_temp_train[:,0:2], (data_temp_train[:,2]).reshape(-1,))
        pred_svm_train = (svm_fit.predict(data_temp_train[:,0:2])).reshape(-1,1)
        error_train_SVM = np.mean(pred_svm_train!=((data_temp_train[:,2]).reshape(-1,1)))
        
        response_categories_train = data_temp_train[:,2]
        response_categories_test = data_temp_test[:,2]
        centres_init = (np.random.uniform(llim,ulim,2*K)).reshape(K,-1)
        final_centres = lloyd_algorithm(data_temp_train[:,0:2],centres_init)
        cond = np.all(final_centres==1)
        
    pred_svm = (svm_fit.predict(data_temp_test[:,0:2])).reshape(-1,1)
    error_test_SVM = np.mean(pred_svm!=((data_temp_test[:,2]).reshape(-1,1)))
    final_weights = rbf_kerel_weights(data_temp_train[:,0:2],response_categories_train,gamma,final_centres)
    error_test_rbf = pred_rbf_kernel(data_temp_test[:,0:2],response_categories_test,final_centres,final_weights,gamma)
    
    if error_test_rbf>error_test_SVM:
        SVM_better_than_rbf = SVM_better_than_rbf + 1
SVM_better_than_rbf/iters

1.0

In [326]:
# Problem 16: Comparing K=9 with K=12 for RBF
iters=10000
Error9_rbf = []
Error12_rbf = []
no_k = [9,12]
gamma = 1.5
for K in no_k:
    for i in range(iters):
        # Generating the Dataset for ith iteration
        cond=True
        while cond==True:
            data_temp_train = gen_data_rbf(llim,ulim,N)
            data_temp_test = gen_data_rbf(llim,ulim,N)
            response_categories_train = data_temp_train[:,2]
            response_categories_test = data_temp_test[:,2]
            centres_init = (np.random.uniform(llim,ulim,2*K)).reshape(K,-1)
            final_centres = lloyd_algorithm(data_temp_train[:,0:2],centres_init)
            cond = np.all(final_centres==1)

        final_weights = rbf_kerel_weights(data_temp_train[:,0:2],response_categories_train,gamma,final_centres)
        error_test_rbf = pred_rbf_kernel(data_temp_test[:,0:2],response_categories_test,final_centres,final_weights,gamma)
        error_train_rbf = pred_rbf_kernel(data_temp_train[:,0:2],response_categories_train,final_centres,final_weights,gamma)
        if K==9:
            Error9_rbf.append((error_train_rbf,error_test_rbf))
        else:
            Error12_rbf.append((error_train_rbf,error_test_rbf))


In [327]:
Einin = []
Eoutout = []
for j in range(iters):
    Einin.append(Error9_rbf[j][0] - Error12_rbf[j][0])
    Eoutout.append((Error9_rbf[j])[1] - (Error12_rbf[j])[1])

m = 0
n = 0
o=0
p=0
q=0
s=0
for l in range(iters):
    if Einin[l]<0 and Eoutout[l]>0: # Ein increases but Eout decreases
        m = m+1
    elif Einin[l]>0 and Eoutout[l]<0: # Ein decreases but Eout increases
        n = n+1
    elif Einin[l]>0 and Eoutout[l]>0: # Both Ein and Eout decrease
        o = o + 1
    elif Einin[l]<0 and Eoutout[l]<0: # Both Ein and Eout increase
        p = p + 1
    elif Einin[l]==0 and Eoutout[l]==0: # Both Ein and Eout stay constant
        q = q + 1  
    if Error9_rbf[j][0]==0: # number of times Ein=0 for kernel with K=9 and gamma=1.5
        s = s + 1
print("Number of times Ein increases and Eout decreases",m)
print("Number of times for which Ein decreases and Eout increases",n)
print("Number of times for which both Ein and Eout decrease",o)
print("Number of times for which Ein and Eout increase",p)
print("Number of times for which Ein and Eout stay the same",q)
print("Number of times for which Ein=0 for K=9 RBF kernel",s)

Number of times Ein increases and Eout decreases 2368
Number of times for which Ein decreases and Eout increases 2265
Number of times for which both Ein and Eout decrease 2333
Number of times for which Ein and Eout increase 2249
Number of times for which Ein and Eout stay the same 12
Number of times for which Ein=0 for K=9 RBF kernel 0


In [328]:
# Problem 17: Comparing gamma=1.5 with gamma=2 for RBF with K=9
iters=10000
gamma15_rbf = []
gamma2_rbf = []
K=9
gamma_vec = [1.5,2]
for gamma in gamma_vec:
    for i in range(iters):
        # Generating the Dataset for ith iteration
        cond=True
        while cond==True:
            data_temp_train = gen_data_rbf(llim,ulim,N)
            data_temp_test = gen_data_rbf(llim,ulim,N)
            response_categories_train = data_temp_train[:,2]
            response_categories_test = data_temp_test[:,2]
            centres_init = (np.random.uniform(llim,ulim,2*K)).reshape(K,-1)
            final_centres = lloyd_algorithm(data_temp_train[:,0:2],centres_init)
            cond = np.all(final_centres==1)

        final_weights = rbf_kerel_weights(data_temp_train[:,0:2],response_categories_train,gamma,final_centres)
        error_test_rbf = pred_rbf_kernel(data_temp_test[:,0:2],response_categories_test,final_centres,final_weights,gamma)
        error_train_rbf = pred_rbf_kernel(data_temp_train[:,0:2],response_categories_train,final_centres,final_weights,gamma)
        if gamma==1.5:
            gamma15_rbf.append((error_train_rbf,error_test_rbf))
        else:
            gamma2_rbf.append((error_train_rbf,error_test_rbf))

In [329]:
Einin = []
Eoutout = []
for j in range(iters):
    Einin.append(gamma15_rbf[j][0] - gamma2_rbf[j][0])
    Eoutout.append((gamma15_rbf[j])[1] - (gamma2_rbf[j])[1])

m = 0
n = 0
o=0
p=0
q=0
for l in range(iters):
    if Einin[l]<0 and Eoutout[l]>0: # Ein increases but Eout decreases
        m = m+1
    elif Einin[l]>0 and Eoutout[l]<0: # Ein decreases but Eout increases
        n = n+1
    elif Einin[l]>0 and Eoutout[l]>0: # Both Ein and Eout decrease
        o = o + 1
    elif Einin[l]<0 and Eoutout[l]<0: # Both Ein and Eout increase
        p = p + 1
    elif Einin[l]==0 and Eoutout[l]==0: # Both Ein and Eout stay constant
        q = q + 1   
print("Number of times Ein increases and Eout decreases",m)
print("Number of times for which Ein decreases and Eout increases",n)
print("Number of times for which both Ein and Eout decrease",o)
print("Number of times for which Ein and Eout increase",p)
print("Number of times for which Ein and Eout stay the same",q)

Number of times Ein increases and Eout decreases 2396
Number of times for which Ein decreases and Eout increases 2350
Number of times for which both Ein and Eout decrease 2229
Number of times for which Ein and Eout increase 2300
Number of times for which Ein and Eout stay the same 15
